In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('../input/smart-agricultural-production-optimizing-engine/Crop_recommendation.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
df['label'].unique()

In [ ]:
df['label'].value_counts()

In [ ]:
import matplotlib.pyplot as plt

# Assuming df is your DataFrame and 'label' is the column of interest
label_counts = df['label'].value_counts()

# Plotting the bar graph
plt.figure(figsize=(8, 6))
label_counts.plot(kind='bar', color='skyblue')

# Adding labels and title
plt.title('Value Counts of Labels')
plt.xlabel('Labels')
plt.ylabel('Count')

# Display the graph
plt.show()


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
df['label'] = encoder.fit_transform(df['label'])

In [ ]:
x = df.drop(['label'], axis=1)
y = df[['label']]

In [ ]:
y['label'].value_counts()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D, InputLayer, Reshape, MaxPooling2D, Flatten,Dropout, BatchNormalization
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

# Output shapes of each set
print("Train set:", x_train.shape, y_train.shape)
print("Validation set:", x_val.shape, y_val.shape)
print("Test set:", x_test.shape, y_test.shape)


In [ ]:
x_train.shape

In [ ]:
model=Sequential()
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(22, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(x_train,y_train,validation_data=(x_val,y_val), epochs=100,verbose=1)

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_accuracy:.2f}')
print(f'Test Loss: {test_loss:.2f}')

In [ ]:
model.save('my_model')

In [ ]:
import tensorflow as tf

# Load the saved model
saved_model_dir = 'my_model'

# Convert the model to TensorFlow Lite format with TensorFlow Select ops enabled (in case of unsupported ops)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,  # Use TensorFlow Lite ops
                                       tf.lite.OpsSet.SELECT_TF_OPS]    # Use TensorFlow ops (if needed)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a .tflite file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model successfully converted to TensorFlow Lite")

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the quantized model to a file
with open('model_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

print("Quantized model successfully saved")


In [ ]:
import numpy as np
import tensorflow as tf

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="model_quantized.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Example input data (replace with your actual input)
# Convert input to float64 to match the model's expected input type.
#107, 34, 32, 26.774637, 66.413269, 6.780064, 177.774507
#64, 32, 6.5, 23.5, 85.4, 5.8, 200
input_data = np.array([[107, 34, 32, 26.774637, 66.413269, 6.780064, 177.774507]], dtype=np.float64)

# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run the interpreter
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Print the output predictions
print("Predicted probabilities: ", output_data)

# If you want the top 4 crops by probability
top_4_indices = np.argsort(output_data[0])[-4:][::-1]
print("Top 4 crop indices:", top_4_indices)
